# Agenda

1. What is Polars? What is its relationship with Pandas and the rest of the data ecosystem in Python?
2. Series
3. Data frames
4. Reading in CSV files
5. dtypes
6. Expressions
7. Selecting rows with `df.select`
8. Selecting columns
9. `df.with_columns`
10. `df.filter`
11. Sorting
12. Grouping
13. Optimizing queries and "lazy frames"